<a href="https://colab.research.google.com/github/componavt/LLLE-R1900s/blob/main/src/visualization/radial_loan_stars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌐 Radial Loan Profiles: Bilingual Radar Charts per Settlement

Each village is a star — rays = credit categories.  
**Ray length** = total money borrowed.  
**Label at tip** = number of loans.

✨ **Features:**
- 🌍 Choose language: Russian (`ru`) or English (`en`)
- 🏘️ Settlement name: `Russian` or `English` from `society_settlement.csv`
- 📏 Axis labels: `loan_short_ru` (RU) or `Name` (EN) from `credit_items.csv`
- 🔢 Loan count shown as number
- 🖼️ Rendered directly in Colab

⚙️ Configurable via `USE_LANGUAGE` variable  
✅ Fully bilingual visualization

⚙️ [2] Install Dependencies & Set Language

In [17]:
# Install compatible versions for static image rendering in Colab
#!pip install -q python-dotenv pandas plotly>=6.1.1 kaleido
!pip install -q python-dotenv pandas plotly

# Clone the repo if running in Colab (optional — if data not uploaded manually)
import os

if not os.path.exists('LLLE-R1900s'):
    !git clone https://github.com/componavt/LLLE-R1900s.git
    %cd LLLE-R1900s
else:
    %cd LLLE-R1900s

# === LANGUAGE SWITCH ===
USE_LANGUAGE = "ru"
assert USE_LANGUAGE in ("ru", "en"), "USE_LANGUAGE must be 'ru' or 'en'"

Cloning into 'LLLE-R1900s'...
remote: Enumerating objects: 531, done.
remote: Counting objects: 100% (531/531), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 531 (delta 398), reused 398 (delta 295), pack-reused 0 (from 0)
Receiving objects: 100% (531/531), 1.39 MiB | 13.33 MiB/s, done.
Resolving deltas: 100% (398/398), done.
/content/LLLE-R1900s/LLLE-R1900s/LLLE-R1900s


📥 [3] Load & Prepare Data with Quadrant Mapping

In [18]:
import os
import pandas as pd
from dotenv import load_dotenv

# Load configuration
load_dotenv('config.env')

# Paths
csv_out_dir = os.getenv('CSV_OUT_DIR', 'data/csv_out')
output_file_name = os.getenv('OUTPUT_CSV_FILE')

if not output_file_name:
    csv_files = [f for f in os.listdir(csv_out_dir) if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError("No CSV files found in the output directory.")
    output_file_name = csv_files[0]

csv_path = os.path.join(csv_out_dir, output_file_name)
print(f"Loading loan data from: {csv_path}")

# Load main loan data
df_loans = pd.read_csv(csv_path)
print(f"Loaded {len(df_loans)} loan records.")

# Load credit items
df_credit = pd.read_csv('data/credit_items.csv')
print(f"Loaded {len(df_credit)} credit item definitions.")

# Load settlement metadata
df_society = pd.read_csv('data/society_settlement.csv')
print(f"Loaded {len(df_society)} settlements.")

# --- Build settlement display name map ---
if USE_LANGUAGE == "ru":
    df_society['display_settlement'] = df_society['Russian']
else:
    df_society['display_settlement'] = df_society['English']

settlement_name_map = dict(zip(df_society['English'], df_society['display_settlement']))

# Validate settlements
unknown_settlements = set(df_loans['settlement']) - set(df_society['English'])
if unknown_settlements:
    print("⚠️ Warning: Unknown settlements in loan data:", unknown_settlements)

# --- Build credit item display label map ---
if USE_LANGUAGE == "ru":
    df_credit['display_label'] = df_credit['loan_short_ru']
else:
    df_credit['display_label'] = df_credit['Name']

label_map = dict(zip(df_credit['Name'], df_credit['display_label']))

# Validate credit items
unknown_items = set(df_loans['credit_item']) - set(df_credit['Name'])
if unknown_items:
    print("⚠️ Warning: Unknown credit_item values:", unknown_items)

# Add display settlement name to loan data
df_loans['display_settlement'] = df_loans['settlement'].map(settlement_name_map)
df_loans = df_loans.dropna(subset=['display_settlement'])

# --- DO NOT collapse loan_type or aci_status ---
# Keep original values: they will define the 3x3 matrix
# Ensure all expected values are present
expected_loan_types = {'productive', 'conditionally_productive', 'non_productive'}
expected_aci_statuses = {'aci', 'conditionally_aci', 'non_aci'}

actual_lt = set(df_credit['loan_type'].dropna().unique())
actual_aci = set(df_credit['aci_status'].dropna().unique())

if not actual_lt.issubset(expected_loan_types):
    print("⚠️ Unexpected loan_type values:", actual_lt - expected_loan_types)
if not actual_aci.issubset(expected_aci_statuses):
    print("⚠️ Unexpected aci_status values:", actual_aci - expected_aci_statuses)

# Create a key for the 3x3 matrix: (loan_type, aci_status)
df_credit['profile_key'] = df_credit['loan_type'] + " | " + df_credit['aci_status']

# For consistent ordering: sort by loan_type then aci_status
loan_type_order = ['productive', 'conditionally_productive', 'non_productive']
aci_status_order = ['aci', 'conditionally_aci', 'non_aci']

# Create all 9 combinations
all_profile_keys = [
    lt + " | " + aci
    for lt in loan_type_order
    for aci in aci_status_order
]

# Map credit item → profile_key
credit_to_profile = dict(zip(df_credit['Name'], df_credit['profile_key']))

# Add profile_key to loans
df_loans['profile_key'] = df_loans['credit_item'].map(credit_to_profile)

# --- Prepare sorted credit item list for radar (optional: group by profile) ---
# We'll sort credit items by (loan_type, aci_status, Name)
df_credit_sorted = df_credit.copy()
df_credit_sorted['lt_order'] = df_credit_sorted['loan_type'].map({lt: i for i, lt in enumerate(loan_type_order)})
df_credit_sorted['aci_order'] = df_credit_sorted['aci_status'].map({aci: i for i, aci in enumerate(aci_status_order)})
df_credit_sorted = df_credit_sorted.sort_values(['lt_order', 'aci_order', 'Name'])

all_credit_items = df_credit_sorted['Name'].tolist()
axis_labels = [label_map[item] for item in all_credit_items]

print("✅ Data prepared with full 3×3 credit profile matrix.")

Loading loan data from: data/csv_out/loans_s28_i21.csv
Loaded 1768 loan records.
Loaded 21 credit item definitions.
Loaded 29 settlements.
✅ Data prepared with full 3×3 credit profile matrix.


🔍 [3.5] Diagnose Full 3×3 Credit Profile Matrix

Diagnose: total loan amount per (loan_type, aci_status) combination — full 3x3 matrix

In [19]:
import os

# Ensure output directory exists
csv_out_dir = os.getenv('CSV_OUT_DIR', 'data/csv_out')
os.makedirs(csv_out_dir, exist_ok=True)

# Diagnose: (1) 3x3 matrix by amount, (2) by count, (3) top items by amount, (4) by count
print("\n" + "="*70)

if USE_LANGUAGE == "ru":
    title_matrix_amount = "📊 МАТРИЦА ПРОФИЛЕЙ КРЕДИТОВ (по сумме), тыс. руб."
    title_matrix_count = "📊 МАТРИЦА ПРОФИЛЕЙ КРЕДИТОВ (по количеству ссуд)"
    row_header = "Тип ссуды \\ Статус РКП"
    legend_text = "✅ Легенда: ✅ = есть данные, – = нет ссуд"
    title_items_amount = "\n\n🏆 ТОП категорий кредитов (по сумме ссуд, тыс. руб.)"
    title_items_count = "\n\n🏆 ТОП категорий кредитов (по количеству ссуд)"
else:
    title_matrix_amount = "📊 CREDIT PROFILE MATRIX (by amount), thsd rub."
    title_matrix_count = "📊 CREDIT PROFILE MATRIX (by loan count)"
    row_header = "Loan Type \\ ACI Status"
    legend_text = "✅ Legend: ✅ = has data, – = no loans"
    title_items_amount = "\n\n🏆 TOP CREDIT CATEGORIES (by loan amount, thsd rub.)"
    title_items_count = "\n\n🏆 TOP CREDIT CATEGORIES (by loan count)"

# === Prepare data safely ===
credit_to_lt = dict(zip(df_credit['Name'], df_credit['loan_type']))
credit_to_aci = dict(zip(df_credit['Name'], df_credit['aci_status']))

df_loans_diag = df_loans.copy()
df_loans_diag['loan_type'] = df_loans_diag['credit_item'].map(credit_to_lt)
df_loans_diag['aci_status'] = df_loans_diag['credit_item'].map(credit_to_aci)

# Aggregate full matrix
loan_type_order = ['productive', 'conditionally_productive', 'non_productive']
aci_status_order = ['aci', 'conditionally_aci', 'non_aci']
all_combinations = pd.DataFrame([
    {'loan_type': lt, 'aci_status': aci}
    for lt in loan_type_order
    for aci in aci_status_order
])

profile_agg = df_loans_diag.groupby(['loan_type', 'aci_status']).agg(
    total_amount_rub=('amount_rubles', 'sum'),
    total_loan_count=('loan_count', 'sum')
).reset_index()

profile_full = all_combinations.merge(
    profile_agg,
    on=['loan_type', 'aci_status'],
    how='left'
).fillna(0)

# Labels for console
LOAN_TYPE_LABELS_RU = {
    "productive": "производственная",
    "conditionally_productive": "условно производственная",
    "non_productive": "непроизводственная"
}
ACI_STATUS_LABELS_RU = {
    "aci": "ркп",
    "conditionally_aci": "условно ркп",
    "non_aci": "не ркп"
}

# Console formatters
def format_thousands_console(amount):
    if amount == 0:
        return "0"
    k = amount / 1000.0
    s = f"{k:.2f}".rstrip('0').rstrip('.')
    return s

def format_count_console(count):
    if count == 0:
        return "0"
    return str(int(count))

# === (1) Matrix by AMOUNT ===
print(title_matrix_amount)
print("="*70)
print(f"\n{row_header:<32}", end="")
for aci in aci_status_order:
    label = ACI_STATUS_LABELS_RU[aci] if USE_LANGUAGE == "ru" else aci.upper()
    print(f"{label:>18}", end="")
print("\n" + "-"*70)

for lt in loan_type_order:
    lt_label = LOAN_TYPE_LABELS_RU[lt] if USE_LANGUAGE == "ru" else lt.replace('_', ' ').title()
    print(f"{lt_label:<32}", end="")
    for aci in aci_status_order:
        row = profile_full[(profile_full['loan_type'] == lt) & (profile_full['aci_status'] == aci)].iloc[0]
        total = row['total_amount_rub']
        formatted = format_thousands_console(total)
        marker = "✅" if total > 0 else "–"
        cell = f"{formatted} {marker}"
        print(f"{cell:>18}", end="")
    print()

print("\n" + "="*70)
print(legend_text)

# === (2) Matrix by COUNT ===
print(title_matrix_count)
print("="*70)
print(f"\n{row_header:<32}", end="")
for aci in aci_status_order:
    label = ACI_STATUS_LABELS_RU[aci] if USE_LANGUAGE == "ru" else aci.upper()
    print(f"{label:>18}", end="")
print("\n" + "-"*70)

for lt in loan_type_order:
    lt_label = LOAN_TYPE_LABELS_RU[lt] if USE_LANGUAGE == "ru" else lt.replace('_', ' ').title()
    print(f"{lt_label:<32}", end="")
    for aci in aci_status_order:
        row = profile_full[(profile_full['loan_type'] == lt) & (profile_full['aci_status'] == aci)].iloc[0]
        count = row['total_loan_count']
        formatted = format_count_console(count)
        marker = "✅" if count > 0 else "–"
        cell = f"{formatted} {marker}"
        print(f"{cell:>18}", end="")
    print()

print("\n" + "="*70)
print(legend_text)

# === (3) Top items by AMOUNT ===
item_agg = df_loans.groupby('credit_item').agg(
    total_amount_rub=('amount_rubles', 'sum'),
    total_loan_count=('loan_count', 'sum')
).reset_index()

# Add display labels
item_agg['name_ru'] = item_agg['credit_item'].map(df_credit.set_index('Name')['loan_short_ru'])
item_agg['name_en'] = item_agg['credit_item'].map(df_credit.set_index('Name')['loan_en'])

# Output by amount
print(title_items_amount)
print("-"*50)
for _, row in item_agg.sort_values('total_amount_rub', ascending=False).iterrows():
    label = row['name_ru'] if USE_LANGUAGE == "ru" else row['name_en']
    amt_k = row['total_amount_rub'] / 1000.0
    if amt_k == 0:
        s = "0"
    else:
        s = f"{amt_k:.2f}".rstrip('0').rstrip('.')
    print(f"{s:>10}  {label}")
print("-"*50)
print(f"Всего категорий: {len(item_agg)}")

# === (4) Top items by COUNT ===
print(title_items_count)
print("-"*50)
for _, row in item_agg.sort_values('total_loan_count', ascending=False).iterrows():
    label = row['name_ru'] if USE_LANGUAGE == "ru" else row['name_en']
    cnt = int(row['total_loan_count'])
    print(f"{cnt:>10}  {label}")
print("-"*50)
print(f"Всего категорий: {len(item_agg)}")

# === SAVE TO CSV (language-agnostic) ===
def clean_float(x):
    return int(x) if x == int(x) else x

# --- (1) Save loan_type_aci_matrix.csv (without total_amount_K_rub) ---
matrix_out = profile_full[['loan_type', 'aci_status', 'total_amount_rub', 'total_loan_count']].copy()
matrix_out['total_amount_rub'] = matrix_out['total_amount_rub'].apply(clean_float)
matrix_out['total_loan_count'] = matrix_out['total_loan_count'].apply(clean_float)

matrix_path = os.path.join(csv_out_dir, 'loan_type_aci_matrix.csv')
matrix_out.to_csv(matrix_path, index=False, encoding='utf-8')
print(f"\n💾 Сохранена матрица типов: {matrix_path}")

# --- (2) Save credit_items_ranking.csv (without ranks and K_rub) ---
ranking_out = item_agg[[
    'credit_item', 'name_ru', 'name_en',
    'total_amount_rub', 'total_loan_count'
]].copy()

ranking_out['total_amount_rub'] = ranking_out['total_amount_rub'].apply(clean_float)
ranking_out['total_loan_count'] = ranking_out['total_loan_count'].apply(clean_float)

ranking_path = os.path.join(csv_out_dir, 'credit_items_ranking.csv')
ranking_out.to_csv(ranking_path, index=False, encoding='utf-8')
print(f"💾 Сохранён рейтинг категорий: {ranking_path}")


📊 МАТРИЦА ПРОФИЛЕЙ КРЕДИТОВ (по сумме), тыс. руб.

Тип ссуды \ Статус РКП                         ркп       условно ркп            не ркп
----------------------------------------------------------------------
производственная                          107.07 ✅          108.03 ✅           79.73 ✅
условно производственная                    1.24 ✅          105.18 ✅            2.18 ✅
непроизводственная                             0 –               0 –          169.49 ✅

✅ Легенда: ✅ = есть данные, – = нет ссуд
📊 МАТРИЦА ПРОФИЛЕЙ КРЕДИТОВ (по количеству ссуд)

Тип ссуды \ Статус РКП                         ркп       условно ркп            не ркп
----------------------------------------------------------------------
производственная                            3038 ✅            2065 ✅            2792 ✅
условно производственная                      50 ✅            1684 ✅              47 ✅
непроизводственная                             0 –               0 –            4899 ✅

✅ Легенда: ✅ = ес

🌟 [4] Individual Settlement Radar Charts (Quadrant Layout)

In [ ]:
# Cell 4: Individual Settlement Radar Charts (Quadrant Layout)
# Comments in English (Per project preference)

import plotly.graph_objects as go
import pandas as pd

# --- Robust preprocessing (assume df_loans, df_credit, df_society and label_map exist from previous cells) ---
# Ensure numeric types for aggregation
if 'df_loans' not in globals():
    raise RuntimeError("df_loans not found. Make sure previous cells (data loading) have been executed.")

# Convert columns to numeric safely
df_loans['amount_rubles'] = pd.to_numeric(df_loans['amount_rubles'], errors='coerce').fillna(0.0)
# 'loan_count' may already be integer but be defensive
df_loans['loan_count'] = pd.to_numeric(df_loans.get('loan_count', 0), errors='coerce').fillna(0).astype(int)

# Rebuild or validate credit item ordering (quadrant-aware)
if 'df_credit_sorted' not in globals():
    df_credit_sorted = df_credit.sort_values(['quadrant', 'Name'])
all_credit_items = df_credit_sorted['Name'].tolist()
axis_labels = [label_map[item] for item in all_credit_items]

# Aggregate loan data by settlement & credit_item
# Use display_settlement if available (created in cell 3); otherwise map now
if 'display_settlement' not in df_loans.columns:
    if 'settlement_name_map' in globals():
        df_loans['display_settlement'] = df_loans['settlement'].map(settlement_name_map)
    else:
        # Fallback: use settlement as-is
        df_loans['display_settlement'] = df_loans['settlement']

# Group and sum
df_agg = df_loans.groupby(['display_settlement', 'credit_item'], dropna=False).agg(
    total_amount=('amount_rubles', 'sum'),
    total_count=('loan_count', 'sum')
).reset_index()

# Unique settlements
settlements = df_agg['display_settlement'].dropna().unique()
shown_count = 0

for settlement in settlements:
    df_set = df_agg[df_agg['display_settlement'] == settlement]

    # Build full vectors (including zeros) in the quadrant-aware order
    amounts = []
    counts = []
    for item in all_credit_items:
        match = df_set[df_set['credit_item'] == item]
        if not match.empty:
            amt = float(match['total_amount'].iloc[0])
            cnt = int(match['total_count'].iloc[0])
        else:
            amt = 0.0
            cnt = 0
        amounts.append(amt)
        counts.append(cnt)

    # Skip entirely empty settlement
    if sum(amounts) == 0:
        continue

    # Close the loop for polar chart
    theta = axis_labels + [axis_labels[0]]
    r = amounts + [amounts[0]]
    text_labels = [str(c) if c > 0 else '' for c in counts] + ['']

    # Create figure
    fig = go.Figure()

    # Filled polygon showing amounts
    fig.add_trace(go.Scatterpolar(
        r=r,
        theta=theta,
        fill='toself',
        name='Total',
        line=dict(color='steelblue', width=1),
        opacity=0.6,
        hoverinfo='none',
        showlegend=False
    ))

    # Markers + text: number of loans at tip of each ray
    # Use a separate marker trace so layout options don't conflict
    fig.add_trace(go.Scatterpolar(
        r=r,
        theta=theta,
        mode='markers+text',
        text=text_labels,
        textposition='top center',
        marker=dict(size=8),
        hovertemplate=['%{theta}<br>Amount: %{r}<br>Loans: '+t for (t,) in zip(text_labels)],
        showlegend=False
    ))

    title = f"Loan Profile: {settlement}" if USE_LANGUAGE == "en" else f"Профиль ссуд: {settlement}"
    radial_title = "Total Amount (Rubles)" if USE_LANGUAGE == "en" else "Сумма ссуд (руб.)"

    fig.update_layout(
        title=title,
        polar=dict(
            radialaxis=dict(visible=True, title=radial_title),
            angularaxis=dict(direction="clockwise")
        ),
        showlegend=False,
        width=700,
        height=700,
        font=dict(size=10)
    )

    # Render in Colab
    fig.show(renderer="colab")
    shown_count += 1

print(f"\n🎉 Displayed {shown_count} quadrant-organized radar charts in {'Russian' if USE_LANGUAGE == 'ru' else 'English'}.")


🎉 Displayed 28 quadrant-organized radar charts in Russian.


📅 [5] Time-Stacked Radar Chart with Quadrant Labels (All Settlements, by Year)

📓 Ячейка 5: Общая радиальная диаграмма по всем поселениям

In [ ]:
# Cell 5: Time-Stacked Radar Chart with Quadrant Labels (All Settlements, by Year)
# Revised implementation: draw **stacked radial segments per axis** (per year increments) and
# use quadrant-level palettes. Labels for quadrants are built from constants and placed
# using LABEL_RADIUS_MULTIPLIER and QUADRANT_ANGLE_OFFSETS_DEG (degrees).

import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

# --- Parameters / constants requested by user ---
LABEL_RADIUS_MULTIPLIER = 1.15  # multiplier to apply to max radius for quadrant labels
QUADRANT_ANGLE_OFFSETS_DEG = {1: -8, 2: 5, 3: 0, 4: 12}

# Bilingual constants (loan_type and aci_status parts)
LOAN_TYPE_PROD = {"ru": "производственная", "en": "Productive"}
LOAN_TYPE_NONPROD = {"ru": "непроизводственная", "en": "Non-productive"}
ACI_STATUS_RKP = {"ru": "ркп", "en": "ACI"}
ACI_STATUS_NONRKP = {"ru": "не ркп", "en": "Non-ACI"}

# Build quadrant label strings using <br> for line breaks (Plotly-compatible)
if USE_LANGUAGE == 'ru':
    QUADRANT_LABELS = {
        1: LOAN_TYPE_PROD['ru'] + '<br>' + ACI_STATUS_RKP['ru'],
        2: LOAN_TYPE_NONPROD['ru'] + '<br>' + ACI_STATUS_RKP['ru'],
        3: LOAN_TYPE_NONPROD['ru'] + '<br>' + ACI_STATUS_NONRKP['ru'],
        4: LOAN_TYPE_PROD['ru'] + '<br>' + ACI_STATUS_NONRKP['ru']
    }
else:
    QUADRANT_LABELS = {
        1: LOAN_TYPE_PROD['en'] + '<br>' + ACI_STATUS_RKP['en'],
        2: LOAN_TYPE_NONPROD['en'] + '<br>' + ACI_STATUS_RKP['en'],
        3: LOAN_TYPE_NONPROD['en'] + '<br>' + ACI_STATUS_NONRKP['en'],
        4: LOAN_TYPE_PROD['en'] + '<br>' + ACI_STATUS_NONRKP['en']
    }

# --- Preconditions ---
if 'df_loans' not in globals():
    raise RuntimeError("df_loans not found. Run data loading cell first.")

# Ensure numeric
df_loans['amount_rubles'] = pd.to_numeric(df_loans['amount_rubles'], errors='coerce').fillna(0.0)

# Prepare credit items and quadrant mapping
if 'df_credit_sorted' not in globals():
    df_credit_sorted = df_credit.sort_values(['quadrant', 'Name'])
all_credit_items = df_credit_sorted['Name'].tolist()
axis_labels = [label_map[item] for item in all_credit_items]
item_to_quadrant = dict(zip(df_credit['Name'], df_credit['quadrant']))
quadrants = [item_to_quadrant[item] for item in all_credit_items]

# Map item indices belonging to each quadrant (ordered)
quad_indices = {}
for q in [1,2,3,4]:
    quad_indices[q] = [i for i, it in enumerate(all_credit_items) if item_to_quadrant[it] == q]

# Aggregate yearly raw amounts per credit_item
df_yearly = df_loans.groupby(['year', 'credit_item'], dropna=False).agg(total_amount=('amount_rubles', 'sum')).reset_index()
years = sorted(df_yearly['year'].dropna().unique())
if not years:
    raise RuntimeError("No year data found in df_loans.")

# Build per-year incremental vectors in the order all_credit_items
yearly_amounts = {year: [] for year in years}
for year in years:
    for item in all_credit_items:
        amt = df_yearly.loc[(df_yearly['year'] == year) & (df_yearly['credit_item'] == item), 'total_amount'].sum()
        yearly_amounts[year].append(float(amt))

# Build cumulative vectors to help compute r_prev, r_curr
cumulative_by_year = []
r_prev = [0.0] * len(all_credit_items)
for year in years:
    r_curr = [r_prev[i] + yearly_amounts[year][i] for i in range(len(all_credit_items))]
    cumulative_by_year.append(r_curr)
    r_prev = r_curr

# Compute global max for scaling and label radius
max_total = max((max(vec) if vec else 0.0) for vec in cumulative_by_year)
if max_total == 0:
    max_total = 1.0
label_radius = max_total * LABEL_RADIUS_MULTIPLIER

# Prepare axis angle mapping (degrees) for label placement and offsets
N = len(all_credit_items)
angle_deg_per_item = {i: (360.0 * i / N) for i in range(N)}
# Helper to get mid-angle (deg) for a segment of indices

def segment_mid_angle_deg(indices):
    if not indices:
        return 0.0
    # Mid between first and last (taking wrap into account)
    first = angle_deg_per_item[indices[0]]
    last = angle_deg_per_item[indices[-1]]
    # If segment wraps beyond 360, adjust (not expected here since indices are contiguous in sorted list)
    mid = (first + last) / 2.0
    return mid

# Generate quadrant palettes (sampled from continuous scales) — one colorscale per quadrant
quad_base_scales = {
    1: 'Greens',
    2: 'Blues',
    3: 'Oranges',
    4: 'Purples'
}
quad_palettes = {}
for q in [1,2,3,4]:
    if len(years) == 1:
        quad_palettes[q] = [px.colors.sample_colorscale(quad_base_scales[q], 0.5)[0]]
    else:
        samples = [i / (len(years)-1) for i in range(len(years))]
        quad_palettes[q] = px.colors.sample_colorscale(quad_base_scales[q], samples)

# Build figure and draw **incremental** rings per year per quadrant (so each ring segment covers only [r_prev, r_curr])
fig = go.Figure()

# Precompute theta_closed for full axis order (categorical labels for hover consistency)
theta_full = axis_labels + [axis_labels[0]]

# For each year, we will draw for each quadrant a polygon that covers only the axes in that quadrant
r_prev = [0.0] * len(all_credit_items)
for yi, year in enumerate(years):
    r_curr = cumulative_by_year[yi]

    # For each quadrant, build polygon over only that quadrant's indices
    for q in [1,2,3,4]:
        indices = quad_indices[q]
        if not indices:
            continue

        # Build outer boundary (r_curr) and inner boundary (r_prev) for these indices
        outer = [r_curr[i] for i in indices]
        inner = [r_prev[i] for i in indices]

        # Close each boundary by appending the first element at the end
        outer_closed = outer + [outer[0]]
        inner_closed = inner + [inner[0]]

        # Build theta arrays for these indices (categorical labels) and close them
        theta_q = [axis_labels[i] for i in indices]
        theta_closed = theta_q + [theta_q[0]]

        # Build ring polygon: outer (forward) then inner (reversed)
        r_ring = outer_closed + inner_closed[::-1]
        theta_ring = theta_closed + theta_closed[::-1]

        # Fill color from quadrant palette for this year
        fillcol = quad_palettes[q][yi]

        fig.add_trace(go.Scatterpolar(
            r=r_ring,
            theta=theta_ring,
            mode='lines',
            fill='toself',
            fillcolor=fillcol,
            line=dict(color=fillcol, width=0.5),
            opacity=0.7,
            name=f"{year} (Q{q})",
            hoverinfo='text',
            text=[f"{axis_labels[i]}<br>Year: {year}<br>Amount outer: {outer[j]}<br>Amount inner: {inner[j]}" \
                  for j,i in enumerate(indices)] + [f"{theta_q[0]}"],
            showlegend=(q==1)  # show legend per year only for first quadrant to avoid clutter
        ))

    r_prev = r_curr.copy()

# Add quadrant labels positioned using mid-angle and user-provided offsets
label_thetas = []
label_rs = []
label_texts = []
for q in [1,2,3,4]:
    indices = quad_indices[q]
    if not indices:
        continue
    mid_deg = segment_mid_angle_deg(indices)
    offset = QUADRANT_ANGLE_OFFSETS_DEG.get(q, 0)
    theta_deg = mid_deg + offset
    # Convert to Plotly's categorical identifier: we'll use degrees converted to corresponding label via index
    # Find nearest axis index to theta_deg
    # Normalize
    theta_deg_norm = theta_deg % 360.0
    # Find closest item index by minimal angular distance
    closest_idx = min(range(N), key=lambda i: abs(((angle_deg_per_item[i] - theta_deg_norm + 180) % 360) - 180))
    # Use the label at closest_idx for theta placement
    theta_label = axis_labels[closest_idx]

    label_thetas.append(theta_label)
    label_rs.append(label_radius)
    label_texts.append(QUADRANT_LABELS[q])

fig.add_trace(go.Scatterpolar(
    r=label_rs,
    theta=label_thetas,
    mode='text',
    text=label_texts,
    textfont=dict(size=12, color='black'),
    showlegend=False
))

# Layout
title = "Сводный профиль ссуд по годам и квадрантам" if USE_LANGUAGE == "ru" else "Combined Loan Profile by Year & Quadrants"
radial_title = "Сумма ссуд (руб.)" if USE_LANGUAGE == "ru" else "Loan Amount (Rubles)"

fig.update_layout(
    title=title,
    polar=dict(
        radialaxis=dict(visible=True, title=radial_title, range=[0, max_total * 1.25]),
        angularaxis=dict(direction='clockwise', tickfont=dict(size=9))
    ),
    showlegend=True,
    legend=dict(orientation='v', yanchor='top', y=0.99, xanchor='left', x=1.02, font=dict(size=11)),
    width=1000,
    height=850,
    font=dict(size=10)
)

fig.show(renderer='colab')
print(f"✅ Time-stacked (incremental segments per axis) radar chart with quadrant palettes for {len(years)} years.")

KeyError: 'quadrant'